In [1]:
import HelperFuncs as hfuncs
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils.data_utils import Sequence
import h5py
import os

Using TensorFlow backend.


Define a class that allows us to sequentially load entire scans. 

In [2]:
BATCH_SIZE = 2
FINAL_WIDTH = 400
FINAL_HEIGHT = 600
CHANNELS = 1
ZONES = 17
ANGLES = 16
#Create directories for sequencer function if they don't exist
for d in ['temp/train_scan/','temp/test_scan/','temp/val_scan/']:
    if not os.path.isdir(d):
        print("Created directory: {}".format(d))
        os.makedirs(d)
        
class ScanSequencer(Sequence):
    idx_dict={}
    
    def __init__(self,num_batches,bucket_name,mode="train_scan"):
        self.num_batches = num_batches
        self.bucket_name = bucket_name
        self.mode = mode
        self.key_id, self.secret_key = hfuncs.GetAWSCredentials()
        self.mode = mode
        self.angles = np.arange(0,64,64//ANGLES)
    def __len__(self):
        return self.num_batches
    def on_epoch_end(self):
        pass
    def __getitem__(self,idx):
        #Get Client
        client = hfuncs.GetAWSClient(self.key_id,self.secret_key)
        bucket = client.Bucket(self.bucket_name)
        
        #Initialize vectors
        X_train = np.zeros((BATCH_SIZE,ANGLES,FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))
        y_train = np.zeros((BATCH_SIZE,ZONES))
        
        j=0
        for i in range(idx*BATCH_SIZE,(idx+1)*BATCH_SIZE):
            #Download batch at index
            path = "temp/{}/batch_{}.hdf5".format(self.mode,i)
            key = "{}/batch_{}.hdf5".format(self.mode,i)
            bucket.download_file(Key=key,Filename=path)
        
            f = h5py.File(path,"r")
            try:
                X_train[j,:,:,:,:] = f['/image'].value[self.angles,:,:,:]
                y_train[j,:] = f['/labels'].value
                j += 1
            finally:
                f.close()
                os.remove(path) 
        return X_train, y_train
         



In [3]:
from twilio.rest import Client
import configparser
from keras.callbacks import Callback

class SMSNotifier(Callback):
    def on_epoch_end(self,epoch,logs=None):
        #Execute every other epoch
        if epoch % 2 == 0:
            #Get config credentials
            config = configparser.ConfigParser()
            config.read('twilio.conf')
            account_sid = config['DEFAULT']['AccountID']
            auth_token = config['DEFAULT']['AuthToken']
            #Get client
            client = Client(account_sid, auth_token)
            #Create message
            if logs is not None:
                message = "Epoch {} complete. Loss: {} Val_loss: {} ".format(epoch,
                                                                             logs.get('loss'),
                                                                             logs.get('val_loss'))
            else:
                message = "Epoch {} complete. No loss data available.".format(epoch)
            #Sendmessage
            message = client.messages.create(
                to="+16178884129", 
                from_="+18572142288",
                body=message)
        else:
            pass

Custom loss to take care of class imbalance. 

In [4]:
from keras import backend as K
import tensorflow as tf

def weighted_binary_crossentropy(y_true, y_pred):
    x = 0.10 #Approximate percentage of positives in each of the 17 zones
    POS_ADJ = 0.5/x
    NEG_ADJ = 0.5/(1-x)
    n_values = BATCH_SIZE
    elems = (tf.unstack(y_true,num=n_values,axis=0)) 
    adj = tf.map_fn(lambda x:tuple([tf.cond(tf.equal(x[i],1.),lambda:POS_ADJ,lambda: NEG_ADJ) for i in range(n_values)]),
                    elems, 
                    dtype = tuple([tf.float32 for i in range(n_values)]) )
    adj = tf.stack(adj,axis=0)
    return K.mean(tf.multiply(adj,K.binary_crossentropy(y_true,y_pred)),axis=-1)

In [5]:
import h5py
from keras.models import load_model

#Load trained model
BASE_MODEL = load_model("check_points/2017_9_25_4_singleSGD_00-0.75.hdf5",custom_objects={'weighted_binary_crossentropy':weighted_binary_crossentropy})

Look at layers 19 and 20 to see where we should begin our new model.

In [8]:
for i in range(-21,0):
    print("Layer{}".format(i))
    print(BASE_MODEL.layers[i].get_config())


Layer-21
{'arguments': {'scale': 1.0}, 'function_type': 'lambda', 'output_shape_type': 'raw', 'name': 'block8_5', 'trainable': True, 'output_shape': (11, 11, 896), 'function': ('ã\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00S\x00\x00\x00s\x14\x00\x00\x00|\x00\x00d\x01\x00\x19|\x00\x00d\x02\x00\x19|\x01\x00\x14\x17S)\x03Né\x00\x00\x00\x00é\x01\x00\x00\x00©\x00)\x02Ú\x06inputsÚ\x05scaler\x03\x00\x00\x00r\x03\x00\x00\x00ú[/home/ubuntu/machine-learning/projects/capstone/DHSThreatRecognition/inception_resnet_v1.pyÚ\x08<lambda>\x9a\x00\x00\x00ó\x00\x00\x00\x00', None, None)}
Layer-20
{'trainable': True, 'data_format': 'channels_last', 'name': 'global_max_pooling2d_2'}
Layer-19
{'trainable': True, 'rate': 0.25, 'name': 'dropout_2'}
Layer-18
{'bias_regularizer': None, 'kernel_constraint': None, 'kernel_initializer': {'class_name': 'VarianceScaling', 'config': {'mode': 'fan_avg', 'scale': 1.0, 'seed': None, 'distribution': 'uniform'}}, 'units': 1, 'trainable': True, 'bias_con

In [10]:
print(BASE_MODEL.layers[-19].output_shape)

(None, 896)


We know from above that our base model will output 896 dimensions. 

In [6]:
BASE_OUTPUT_DIM = BASE_MODEL.layers[-19].output_shape[1]

In [6]:
#Load validation data on memory so that we can visualize the weights.
UPLOAD_BUCKET = 'cleandhsdata' #bucket where clean data was stored
key_id, secret_key = hfuncs.GetAWSCredentials()
client = hfuncs.GetAWSClient(key_id,secret_key)
bucket = client.Bucket(UPLOAD_BUCKET)

#Initialize validation sequencer
mode = "val_scan"
num_batches = sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1 #train,test,val root directories have their own keys
val_seq = ScanSequencer(num_batches,UPLOAD_BUCKET,mode=mode)
size =5
#Val samples 1 and 2 were accidentally overwritten so get second batch
X_val, y_val = val_seq.__getitem__()
    


ValueError: could not broadcast input array from shape (20,8,400,600,1) into shape (8,400,600,1)

In [9]:
from keras.layers import Input,Flatten,Dense,Concatenate,Dropout,concatenate
from keras.models import Model
from datetime import datetime
from keras.callbacks import TensorBoard,EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
from keras.optimizers import Adam
from keras.metrics import binary_accuracy
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras.losses import binary_crossentropy
import inception_resnet_v1 as rv1

def trainCNNResRNNAdamComplex(lr,beta1,beta2,lstm_dim,description="RNN_complex"):
    #Build new sequential model,Removing dense layers from model
    print("Creating model...")
    input_scan = Input(shape=(ANGLES,FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))
    input_image = Input(shape=(FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))
    #From our base model, we'll remove 18 extraneous layers (one concatenation and 17 dense) since these
    #were only used for making predictions for each of the zones using the extracted features.  Now, we'll be making 
    #predictions only after we've seen all of the supplied angle slices.
    #new_model_bottom = Model(BASE_MODEL.input,BASE_MODEL.get_layer('dropout_2').output)
    modified_resnet = rv1.InceptionResNetV1(include_top=False,
                      weights=None,
                      input_tensor=input_image,
                      pooling='max',
                     alpha = 1)
    
    sequenced_model = TimeDistributed(modified_resnet)(input_scan)

    #One lstm layer for now
    lstm = LSTM(lstm_dim,recurrent_dropout=0.20)(sequenced_model)

    #Finally, 17 dense layers connected to the output
    output_nodes = []
    for i in range(ZONES):
        output_nodes.append(Dense(1,activation='sigmoid')(lstm))

    out = concatenate(output_nodes)

    #complete model
    recurrent_model = Model(input_scan, out)
   
    #optimizer
    adam = Adam(lr,beta_1=beta1,beta_2=beta2)
    print("Compiling model...")
    recurrent_model.compile(optimizer=adam,
                              metrics=[binary_accuracy,binary_crossentropy],
                             loss= weighted_binary_crossentropy)

    #Tensorboard
    x = datetime.today()
    stamp = "{}-{}-{}_{}:{}:{}_lr-{}_beta1-{}_beta2-{}_lstmDim-{}_ANG-{}_{}".format(x.year,x.month,
                                                         x.day,x.hour,x.minute,
                                                         x.second,lr,beta1,beta2,lstm_dim,ANGLES,description)
    tensorboard = TensorBoard(log_dir="logs/{}".format(stamp),histogram_freq=0,batch_size=BATCH_SIZE,
                              write_grads=False,write_images=False)
    #Model checkpoint
    check_point_dir = 'check_points/'
    if not os.path.isdir(check_point_dir):
        os.makedirs(check_point_dir)    
    chkpt = ModelCheckpoint(os.path.join(check_point_dir,"{}_{}_{}_{}_".format(x.year,x.month,x.day,x.hour) + "ScanModel_{epoch:02d}-{val_loss:.2f}.hdf5"),
                           monitor='val_binary_crossentropy',
                           verbose=1,
                           save_best_only=True)
    #Reduce learning rate on plateau
    reduce_lr = ReduceLROnPlateau(monitor='val_binary_crossentropy',
                                 factor=0.2,
                                 patience=1,
                                 verbose=1,
                                 min_lr=0.00005,
                                 cooldown = 4)
    #Notifications
    notify = SMSNotifier()
    
    #Early stopping callback
    estop = EarlyStopping(monitor='val_binary_crossentropy',min_delta=0.0001,patience=10)
    
    #Generators and fit
    print("Initializing generators...")
    #Bucket with clean data
    UPLOAD_BUCKET = 'cleandhsdata' #bucket where clean data was stored
    key_id, secret_key = hfuncs.GetAWSCredentials()
    client = hfuncs.GetAWSClient(key_id,secret_key)
    bucket = client.Bucket(UPLOAD_BUCKET)

    #Initialize train sequencer
    mode ="train_scan"
    num_batches_train = (sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1)//BATCH_SIZE #train,test,val root directories have their own keys
    #num_batches = 660//BATCH_SIZE
    train_seq = ScanSequencer(num_batches_train,UPLOAD_BUCKET,mode=mode)

    #Initialize validation sequencer
    mode = "val_scan"
    num_batches_val = (sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1)//BATCH_SIZE #train,test,val root directories have their own keys
    val_seq = ScanSequencer(num_batches_val,UPLOAD_BUCKET,mode=mode)

    
    
    print("Beginning training...")
    hist = recurrent_model.fit_generator(train_seq,
                                           steps_per_epoch=num_batches_train,
                                           #steps_per_epoch=5,
                                           epochs=100,
                                           validation_data = val_seq,
                                           validation_steps = num_batches_val,
                                           #validation_steps = 5,
                                           callbacks=[tensorboard,chkpt,reduce_lr,notify,estop],
                                          use_multiprocessing =False,workers=1)
    return hist,recurrent_model

from keras.layers import Input,Flatten,Dense,Concatenate,Dropout,concatenate,Activation
from keras.models import Model
from datetime import datetime
from keras.callbacks import TensorBoard,EarlyStopping
from keras.optimizers import Adam
from keras import metrics
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras import backend as K

K.set_learning_phase(1)#Due to some keras bug, flag has to be constant. It works when I use dropout though?

def trainCNNResRNNAdamSimple(lr,beta1,beta2,lstm_dim,description="RNN_simple"):
    #Build new sequential model,Removing dense layers from model
    print("Creating model...")
    input_scan = Input(shape=(ANGLES,FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))
    
    #From our base model, we'll remove 18 extraneous layers (one concatenation and 17 dense) since these
    #were only used for making predictions for each of the zones using the extracted features.  Now, we'll be making 
    #predictions only after we've seen all of the supplied angle slices.    
    new_model_bottom = Model(BASE_MODEL.input,BASE_MODEL.output)
    sequenced_model = TimeDistributed(new_model_bottom)(input_scan)

    #One lstm layer for now
    lstm = LSTM(lstm_dim,recurrent_dropout=0)(sequenced_model)

    #Treat output of LSTM as individual guessess for each of the 17 zones
    

    out = Activation('sigmoid')(lstm)

    #complete model
    recurrent_model = Model(input_scan, out)
   
    #optimizer
    adam = Adam(lr,beta_1=beta1,beta_2=beta2)
    print("Compiling model...")
    recurrent_model.compile(optimizer=adam,
                              metrics=[metrics.binary_accuracy],
                             loss= 'binary_crossentropy')

    #Tensorboard
    x = datetime.today()
    stamp = "{}-{}-{}_{}:{}:{}_lr-{}_beta1-{}_beta2-{}_lstmDim-{}_ANG-{}_{}".format(x.year,x.month,
                                                         x.day,x.hour,x.minute,
                                                         x.second,lr,beta1,beta2,lstm_dim,ANGLES,description)
    tensorboard = TensorBoard(log_dir="logs/{}".format(stamp),histogram_freq=0,batch_size=BATCH_SIZE,
                              write_grads=False,write_images=False)

    #Generators and fit
    print("Initializing generators...")
    #Bucket with clean data
    UPLOAD_BUCKET = 'cleandhsdata' #bucket where clean data was stored
    key_id, secret_key = hfuncs.GetAWSCredentials()
    client = hfuncs.GetAWSClient(key_id,secret_key)
    bucket = client.Bucket(UPLOAD_BUCKET)

    #Initialize train sequencer
    mode ="train_scan"
    num_batches = (sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1)//BATCH_SIZE #train,test,val root directories have their own keys
    train_seq = ScanSequencer(num_batches,UPLOAD_BUCKET,mode=mode)

    #Initialize validation sequencer
    mode = "val_scan"
    num_batches = (sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1)//BATCH_SIZE #train,test,val root directories have their own keys
    val_seq = ScanSequencer(num_batches,UPLOAD_BUCKET,mode=mode)

    #Early stopping callback
    estop = EarlyStopping(monitor='eval_loss',min_delta=0.005,patience=4)
    print("Beginning training...")
    hist = recurrent_model.fit_generator(train_seq,
                                           steps_per_epoch=660//5,
                                           epochs=50,
                                           validation_data = val_seq,
                                           validation_steps = 200//BATCH_SIZE,
                                           callbacks=[tensorboard,estop],
                                          use_multiprocessing =True,workers=1)
    return hist,recurrent_model

from keras.layers import Input,Flatten,Dense,Concatenate,Dropout,concatenate,Activation
from keras.models import Model
from datetime import datetime
from keras.callbacks import TensorBoard,EarlyStopping
from keras.optimizers import Adam
from keras import metrics
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras import backend as K
from keras.layers.merge import Average

K.set_learning_phase(1)#Due to some keras bug, flag has to be constant. It works when I use dropout though?

def trainCNNResAdamAverage(lr,beta1,beta2,lstm_dim,description="RNN_average"):
    #Build new sequential model,Removing dense layers from model
    print("Creating model...")
    input_scan = Input(shape=(ANGLES,FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))
    
    #From our base model, we'll remove 18 extraneous layers (one concatenation and 17 dense) since these
    #were only used for making predictions for each of the zones using the extracted features.  Now, we'll be making 
    #predictions only after we've seen all of the supplied angle slices.    
    new_model_bottom = Model(BASE_MODEL.input,BASE_MODEL.output)
    sequenced_model = TimeDistributed(new_model_bottom)(input_scan)
    
    
    #One lstm layer for now
    out = Average()(sequenced_model)

    #Treat output of LSTM as individual guessess for each of the 17 zones
    

    

    #complete model
    recurrent_model = Model(input_scan, out)
   
    #optimizer
    adam = Adam(lr,beta_1=beta1,beta_2=beta2)
    print("Compiling model...")
    recurrent_model.compile(optimizer=adam,
                              metrics=[metrics.binary_accuracy],
                             loss= 'binary_crossentropy')

    #Tensorboard
    x = datetime.today()
    stamp = "{}-{}-{}_{}:{}:{}_lr-{}_beta1-{}_beta2-{}_lstmDim-{}_ANG-{}_{}".format(x.year,x.month,
                                                         x.day,x.hour,x.minute,
                                                         x.second,lr,beta1,beta2,lstm_dim,ANGLES,description)
    tensorboard = TensorBoard(log_dir="logs/{}".format(stamp),histogram_freq=0,batch_size=BATCH_SIZE,
                              write_grads=False,write_images=False)

    #Generators and fit
    print("Initializing generators...")
    #Bucket with clean data
    UPLOAD_BUCKET = 'cleandhsdata' #bucket where clean data was stored
    key_id, secret_key = hfuncs.GetAWSCredentials()
    client = hfuncs.GetAWSClient(key_id,secret_key)
    bucket = client.Bucket(UPLOAD_BUCKET)

    #Initialize train sequencer
    mode ="train_scan"
    num_batches = (sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1)//BATCH_SIZE #train,test,val root directories have their own keys
    train_seq = ScanSequencer(num_batches,UPLOAD_BUCKET,mode=mode)

    #Initialize validation sequencer
    mode = "val_scan"
    num_batches = (sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1)//BATCH_SIZE #train,test,val root directories have their own keys
    val_seq = ScanSequencer(num_batches,UPLOAD_BUCKET,mode=mode)

    #Early stopping callback
    estop = EarlyStopping(monitor='eval_loss',min_delta=0.005,patience=4)
    print("Beginning training...")
    hist = recurrent_model.fit_generator(train_seq,
                                           steps_per_epoch=660//5,
                                           epochs=100,
                                           validation_data = val_seq,
                                           validation_steps = 200//BATCH_SIZE,
                                           callbacks=[tensorboard,estop],
                                          use_multiprocessing =True,workers=1)
    return hist,recurrent_model

from keras.layers import Input,Flatten,Dense,Concatenate,Dropout,concatenate,Activation
from keras.models import Model
from datetime import datetime
from keras.callbacks import TensorBoard,EarlyStopping
from keras.optimizers import Adam
from keras import metrics
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras import backend as K

K.set_learning_phase(1)#Due to some keras bug, flag has to be constant. It works when I use dropout though?

def trainCNNResRNNAdamSimple(lr,beta1,beta2,lstm_dim,description="RNN_simple"):
    #Build new sequential model,Removing dense layers from model
    print("Creating model...")
    input_scan = Input(shape=(ANGLES,FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))
    
    #From our base model, we'll remove 18 extraneous layers (one concatenation and 17 dense) since these
    #were only used for making predictions for each of the zones using the extracted features.  Now, we'll be making 
    #predictions only after we've seen all of the supplied angle slices.    
    new_model_bottom = Model(BASE_MODEL.input,BASE_MODEL.output)
    sequenced_model = TimeDistributed(new_model_bottom)(input_scan)

    #One lstm layer for now
    lstm = LSTM(lstm_dim,recurrent_dropout=0)(sequenced_model)

    #Treat output of LSTM as individual guessess for each of the 17 zones
    

    out = Activation('sigmoid')(lstm)

    #complete model
    recurrent_model = Model(input_scan, out)
   
    #optimizer
    adam = Adam(lr,beta_1=beta1,beta_2=beta2)
    print("Compiling model...")
    recurrent_model.compile(optimizer=adam,
                              metrics=[metrics.binary_accuracy],
                             loss= 'binary_crossentropy')

    #Tensorboard
    x = datetime.today()
    stamp = "{}-{}-{}_{}:{}:{}_lr-{}_beta1-{}_beta2-{}_lstmDim-{}_ANG-{}_{}".format(x.year,x.month,
                                                         x.day,x.hour,x.minute,
                                                         x.second,lr,beta1,beta2,lstm_dim,ANGLES,description)
    tensorboard = TensorBoard(log_dir="logs/{}".format(stamp),histogram_freq=0,batch_size=BATCH_SIZE,
                              write_grads=False,write_images=False)

    #Generators and fit
    print("Initializing generators...")
    #Bucket with clean data
    UPLOAD_BUCKET = 'cleandhsdata' #bucket where clean data was stored
    key_id, secret_key = hfuncs.GetAWSCredentials()
    client = hfuncs.GetAWSClient(key_id,secret_key)
    bucket = client.Bucket(UPLOAD_BUCKET)

    #Initialize train sequencer
    mode ="train_scan"
    num_batches = (sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1)//BATCH_SIZE #train,test,val root directories have their own keys
    train_seq = ScanSequencer(num_batches,UPLOAD_BUCKET,mode=mode)

    #Initialize validation sequencer
    mode = "val_scan"
    num_batches = (sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1)//BATCH_SIZE #train,test,val root directories have their own keys
    val_seq = ScanSequencer(num_batches,UPLOAD_BUCKET,mode=mode)

    #Early stopping callback
    estop = EarlyStopping(monitor='eval_loss',min_delta=0.005,patience=4)
    print("Beginning training...")
    hist = recurrent_model.fit_generator(train_seq,
                                           steps_per_epoch=660//5,
                                           epochs=100,
                                           validation_data = val_seq,
                                           validation_steps = 200//BATCH_SIZE,
                                           callbacks=[tensorboard,estop],
                                          use_multiprocessing =True,workers=1)
    return hist,recurrent_model

from keras.layers import Input,Flatten,Dense,Concatenate,Dropout,concatenate
from keras.models import Model
from datetime import datetime
from keras.callbacks import TensorBoard,EarlyStopping
from keras.optimizers import Adam
from keras import metrics
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM
K.set_learning_phase(1)#Due to some keras bug, flag has to be constant. It works when I use dropout though?
def trainCNNResRNNAdamComplexIsh(lr,beta1,beta2,lstm_dim,description="RNN_complexISH"):
    #Build new sequential model,Removing dense layers from model
    print("Creating model...")
    input_scan = Input(shape=(ANGLES,FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))
    
    #From our base model, we'll remove 18 extraneous layers (one concatenation and 17 dense) since these
    #were only used for making predictions for each of the zones using the extracted features.  Now, we'll be making 
    #predictions only after we've seen all of the supplied angle slices.
    new_model_bottom = Model(BASE_MODEL.input,BASE_MODEL.output)
    
    sequenced_model = TimeDistributed(new_model_bottom)(input_scan)

    #One lstm layer for now
    lstm = LSTM(lstm_dim)(sequenced_model)

    #Finally, 17 dense layers connected to the output
    output_nodes = []
    for i in range(ZONES):
        output_nodes.append(Dense(1,activation='sigmoid')(lstm))

    out = concatenate(output_nodes)

    #complete model
    recurrent_model = Model(input_scan, out)
   
    #optimizer
    adam = Adam(lr,beta_1=beta1,beta_2=beta2)
    print("Compiling model...")
    recurrent_model.compile(optimizer=adam,
                              metrics=[metrics.binary_accuracy],
                             loss= 'binary_crossentropy')

    #Tensorboard
    x = datetime.today()
    stamp = "{}-{}-{}_{}:{}:{}_lr-{}_beta1-{}_beta2-{}_lstmDim-{}_ANG-{}_{}".format(x.year,x.month,
                                                         x.day,x.hour,x.minute,
                                                         x.second,lr,beta1,beta2,lstm_dim,ANGLES,description)
    tensorboard = TensorBoard(log_dir="logs/{}".format(stamp),histogram_freq=0,batch_size=BATCH_SIZE,
                              write_grads=False,write_images=False)

    #Generators and fit
    print("Initializing generators...")
    #Bucket with clean data
    UPLOAD_BUCKET = 'cleandhsdata' #bucket where clean data was stored
    key_id, secret_key = hfuncs.GetAWSCredentials()
    client = hfuncs.GetAWSClient(key_id,secret_key)
    bucket = client.Bucket(UPLOAD_BUCKET)

    #Initialize train sequencer
    mode ="train_scan"
    num_batches = sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1 #train,test,val root directories have their own keys
    train_seq = ScanSequencer(num_batches,UPLOAD_BUCKET,mode=mode)

    #Initialize validation sequencer
    mode = "val_scan"
    num_batches = sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1 #train,test,val root directories have their own keys
    val_seq = ScanSequencer(num_batches,UPLOAD_BUCKET,mode=mode)

    #Early stopping callback
    estop = EarlyStopping(monitor='val_loss',min_delta=0.005,patience=4)
    print("Beginning training...")
    hist = recurrent_model.fit_generator(train_seq,
                                           steps_per_epoch=660//BATCH_SIZE,
                                           epochs=50,
                                           validation_data = val_seq,
                                           validation_steps = 200//BATCH_SIZE,
                                           callbacks=[tensorboard,estop],
                                          use_multiprocessing =False,workers=1)
    return hist,recurrent_model

Let's try training the model

In [ ]:
hist,recurrent_model = trainCNNResRNNAdamComplex(lr=0.002,beta1=0.9,beta2=0.99,lstm_dim=800)

Creating model...
Compiling model...
Initializing generators...
Beginning training...
Epoch 1/100
445/445 [==============================] - 1415s - loss: 0.7727 - binary_accuracy: 0.5140 - binary_crossentropy: 0.7353 - val_loss: 0.8007 - val_binary_accuracy: 0.4706 - val_binary_crossentropy: 0.7645
Epoch 2/100
445/445 [==============================] - 1379s - loss: 0.7621 - binary_accuracy: 0.5205 - binary_crossentropy: 0.7432 - val_loss: 0.7210 - val_binary_accuracy: 0.5289 - val_binary_crossentropy: 0.6673
Epoch 3/100
445/445 [==============================] - 1358s - loss: 0.7531 - binary_accuracy: 0.5288 - binary_crossentropy: 0.7376 - val_loss: 0.7229 - val_binary_accuracy: 0.6633 - val_binary_crossentropy: 0.6859
Epoch 4/100
444/445 [============================>.] - ETA: 2s - loss: 0.7523 - binary_accuracy: 0.5244 - binary_crossentropy: 0.7373Epoch 00003: val_binary_crossentropy did not improve

Epoch 00003: reducing learning rate to 0.0004000000189989805.
445/445 [===========

In [ ]:
print(os.system("aws ec2 stop-instances --instance-ids i-0172c75d2de9bad71"))

In [ ]:
try:
    hist,recurrent_model = trainCNNResRNNAdamComplex(lr=0.0001,beta1=0.9,beta2=0.99,lstm_dim=800)
    del hist,recurrent_model
except:
    print("SOmething went wrong")
try:
    hist,recurrent_model = trainCNNResAdamAverage(lr=0.0001,beta1=0.9,beta2=0.99,lstm_dim=0)
    del hist,recurrent_model
except:
    print("Something went wrong")                                                  
try:
    hist,recurrent_model = trainCNNResRNNAdamSimple(lr=0.0001,beta1=0.9,beta2=0.999,lstm_dim=17)
    del hist,recurrent_model
except:
    print("Something went wrong") 
try:
    hist,recurrent_model = trainCNNResRNNAdamComplexIsh(lr=0.0001,beta1=0.9,beta2=0.999,lstm_dim=50)
    del hist,recurrent_model
except:
    print("Something went wrong") 
                                                  